In [ ]:
require 'cutorch'
require 'nn'
require 'cunn'
require 'image'
require 'utils'

# Sample from models

We provide pretrained models in `texture_models` directory. As of 02/02/2015 nn.BatchNorm throws a error in CPU mode, so only GPU mode supported. 

You can load any model just by changing the following line.

In [ ]:
texture_name = 'bricks.png'

The full list of textures available (copy the name to the cell above):

pixelcity_windows2.jpg

bricks.png

red-peppers256.o.jpg

pebble.png

In [ ]:
model_name = texture_name .. '_model.t7' 
model = torch.load('texture_models/' .. model_name):cuda() 
function nn.BatchNormalization:checkInputDim(input) 
end

### Here we present original texture followed by 3 samples

In [ ]:
-- noise is generated with special module nn.GenNoise inside the network
-- so using zeros as input is completely fine  
samples = model:forward(torch.zeros(3,3,256,256):cuda()):double()
itorch.image(image.load('texture_models/' .. texture_name))
itorch.image(deprocess(samples))

# Other resolution

In [ ]:
sample_size = 64 -- powers of 2 are supported for now
samples = model:forward(torch.zeros(1,3,sample_size,sample_size):cuda()):double()
print ('sample size is', samples:size(3))
itorch.image(deprocess(samples))

In [ ]:
sample_size = 512 -- powers of 2 are supported for now
samples = model:forward(torch.zeros(1,3,sample_size,sample_size):cuda()):double()
print ('sample size is', samples:size(3))
itorch.image(deprocess(samples))

# Explore texture element size

Here we explore the effect of additional noise. i-th picture generated setting all noise inputs but i-th to zero.

In [ ]:
sample_size = 256 

In [ ]:
texture_name = 'red-peppers256.o.jpg'
model_name1= texture_name .. '_model.t7' 
model = torch.load('texture_models/' .. model_name1):cuda()


noise_modules, _ = model:findModules('nn.GenNoise')
imgs = {}
for i=1,#noise_modules do
    zero_all_but(i,noise_modules)
    
    samples = model:forward(torch.zeros(1,3,sample_size,sample_size):cuda()):double()
    imgs[#noise_modules - i+1] = deprocess(samples)
end
itorch.image(torch.cat(imgs,1))

And here we set 5th and 6th blob to zero and compare it with a sample generated using all noise blobs. Note that difference is negligible.

In [ ]:
for i=1,#noise_modules do
    noise_modules[i].action = 'fix'
end

samples = model:forward(torch.zeros(1,3,sample_size,sample_size):cuda()):double()
print ('Generate, using all noise inputs.')
smaple_all_inputs = deprocess(samples)
itorch.image(deprocess(samples))


noise_modules[1].action = 'zero'
noise_modules[2].action = 'zero'
noise_modules[3].action = 'fix'
noise_modules[4].action = 'fix'
noise_modules[5].action = 'fix'
noise_modules[6].action = 'fix'

samples = model:forward(torch.zeros(1,3,sample_size,sample_size):cuda()):double()
print ('Generate, using only first four.\n')
itorch.image(deprocess(samples))

print ('Diff')
itorch.image(deprocess(samples)-smaple_all_inputs)